In [151]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer 

In [152]:
data = pd.read_table('data.tsv')
A = data['Sentiment']
B = np.bincount(A)
ii = np.nonzero(B)[0]
a = np.array([ii,B[ii]])
print(a.T)

[[    0  7072]
 [    1 27273]
 [    2 79582]
 [    3 32927]
 [    4  9206]]


In [153]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [154]:
stop = stopwords.words('english')

In [155]:
from bs4 import BeautifulSoup 
def filter_punc_stopwrds(txt):
    txtin=BeautifulSoup(txt,'lxml').get_text()
    nopunc=''.join( c for c in stemmer.stem(txtin.lower())  ) 
    return ' '.join( word for word in nopunc.split() if word not in stopwords.words('english'))

In [156]:
data['New'] = data['Phrase'].apply(filter_punc_stopwrds)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [157]:
data['New']

0         series escapades demonstrating adage good goos...
1            series escapades demonstrating adage good goos
2                                                      seri
3                                                          
4                                                      seri
5                   escapades demonstrating adage good goos
6                                                          
7                   escapades demonstrating adage good goos
8                                                   escapad
9                             demonstrating adage good goos
10                                       demonstrating adag
11                                                 demonstr
12                                                     adag
13                                                         
14                                                     adag
15                                                good goos
16                                      

In [161]:
vectorizer = CountVectorizer()
corpus = np.array(data['New'][:5])
X = vectorizer.fit_transform(corpus)
X.toarray()

array([[1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]], dtype=int64)

In [159]:
tfvectorizer = TfidfVectorizer()
corpus = np.array(data['New'][:5])
Y = tfvectorizer.fit_transform(corpus)
Y.toarray()

array([[0.21404559, 0.26530399, 0.26530399, 0.26530399, 0.21404559,
        0.21404559, 0.26530399, 0.42809118, 0.        , 0.26530399,
        0.26530399, 0.26530399, 0.26530399, 0.        , 0.21404559,
        0.26530399],
       [0.39114171, 0.        , 0.        , 0.        , 0.39114171,
        0.39114171, 0.        , 0.39114171, 0.48481007, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.39114171,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 

In [147]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

x_train = data.loc[:12000, 'New'].values
y_train = data.loc[:12000, 'Sentiment'].values
x_test = data.loc[:266, 'New'].values
y_test = data.loc[:266, 'Sentiment'].values


classifier1 = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB(alpha=0.00000001))])

classifier2 = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression( solver='saga',multi_class='multinomial'))])

classifier3 = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(gamma=10,kernel='poly'))])

classifier1.fit(x_train, y_train)
predicted = classifier1.predict(x_test)

classifier2.fit(x_train, y_train)
predicted2 = classifier2.predict(x_test)

classifier3.fit(x_train, y_train)
predicted3 = classifier3.predict(x_test)
# get the accuracy
print ("1. SVM: ", "%.2f" % (accuracy_score(y_test, predicted3)*100), "%")
print ("2. NaiveBayes Multinomial:", "%.2f" % (accuracy_score(y_test, predicted)*100), "%")
print ("3. Logistic Regression: ", "%.2f" % (accuracy_score(y_test, predicted2)*100), "%")



1. SVM:  90.26 %
2. NaiveBayes Multinomial: 80.90 %
3. Logistic Regression:  76.78 %
